# Palavras mais-ditas: Uma ferramenta de identificação e visualização dos termos mais utilizados nos discursos da Câmara dos Deputados
A ideia do código surgiu da tese de que palavras-chaves podem indicar os temas de maior relevância em qualquer debate ou discurso. Como primeiro teste da ferramenta, foram analisadas as palavras mais usadas nos discursos oficiais pelos Deputados Federais no início dos contágios da Covid-19, em Março de 2020. A intenção é utilizar o mesmo código para analisar as palavras-chave de um ano depois, no auge do número de mortes na Pandemia, em Março de 2021.
O texto e o wordcloud gerados facilitarão a compreensão dos temas em destaque e permitirão a análise das palavras mais utilizadas em qualquer discurso na Câmara dos Deputados.

In [ ]:
#Primeiro, instalei as bibliotecas que vou precisar

In [ ]:
! pip3 install PyPDF2
import PyPDF2
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 4.9 MB/s eta 0:00:00


In [ ]:
#Agora preciso de baixar o PDF com os discursos de Março de 2020 da Câmara dos Deputados. Para isso, vou automatizar o processo.

In [ ]:
import requests

In [ ]:
response = requests.get('https://www2.camara.leg.br/atividade-legislativa/discursos-e-notas-taquigraficas/discursos-em-destaque/coronavirus/DiscursossobreCoronavrusMAR2020.pdf')
with open('file.pdf', 'wb') as f:
    f.write(response.content)
    f.close()

In [ ]:
discursos_mar_20='file.pdf' #O arquivo estava no diretório como file.pdf! VAqui renomeei como discursos_mar-20.

In [ ]:
pdfFileObj = open(discursos_mar_20,'rb') #Para definir a abertura do arquivo

In [ ]:
pdfReader = PyPDF2.PdfReader(pdfFileObj) #Acionei a leitura do PDF

In [ ]:
arquivo = PyPDF2.PdfReader(discursos_mar_20, strict=False)

In [ ]:
num_pags = len(arquivo.pages)

In [ ]:
len(arquivo.pages)

526

In [ ]:
#É necessário transformar o PDF em uma lista para extrair palavra por palavra do discurso desejado.

In [ ]:
pontuação = ['(',')',';',':','[',']',',', '!','?', '.'] #Selecionei as pontuações mais usadas no português para não atrapalhar a contabilização dos termos.

In [ ]:
text=[] 
for x in range(0, num_pags):
    page = arquivo.pages[x].extract_text().replace('\n','')
    text.append(page)
text #Aqui funciona a extração de texto por páginas e remoção de quebras de texto.

In [ ]:
#Para dividir o texto em palavras, foi necessário instalar a biblioteca NLTK:

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords') #As stopwords são os termos que "ligam" as palavras chaves às outras, portanto podem ser retirados da contagem

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
palavras_extras = ['O','SR.','Sr.','DISCURSO','Sessão','SUMÁRIO','sumário','Sumário','CÂMARA','DOS','DEPUTADOS','DCD','Extraordinária','CD', '.','-', 'nº','ORDEM', 'É',"'", 'DO','DIA',"``",'E','A','Eu','Publ','Deliberativa']

In [ ]:
#As palavras extras eram palavras usadas nos modelos documentais da Câmaras dos Deputados, e poderiam atrapalhar a filtragem.

In [ ]:
pt_stp_words = stopwords.words('portuguese')
pt_stp_words = pt_stp_words + pontuação + palavras_extras

In [ ]:
word_tokens = word_tokenize(str(text))
word_tokens #Aqui as palavras foram tokenizadas, extraídas e separadas de uma por uma.

In [ ]:
filtered_sentence = [x for x in word_tokens if not x in pt_stp_words] #filtrei as sentenças sem as stopwords e pontuações.

filtered_sentence = []

for word in word_tokens:
  if word not in pt_stp_words:
    filtered_sentence.append(word)

In [ ]:
len(filtered_sentence) #Esta linha ajuda a verificar a quantidade de palavras filtradas, e garantir que o filtro funcionou.

109075

In [ ]:
palavras_unicas = [] #Palavras únicas são as "palavras chave", aquelas que restam após a retirada de stopwords e pontuações.
contagem = {}
for word in filtered_sentence: 
  if word not in palavras_unicas:
    palavras_unicas.append(word)
    contagem[word]=1
  else:
    contagem[word] = contagem[word]+1
contagem

In [ ]:
tabela=pd.Series(contagem) #Aqui foi criada uma tabela para facilitar a visualização e a contagem decrescente de frequência das palavras.
tabela.sort_values(ascending=False).head(50) #Decidi pelas 50 palavras mais frequentes, mas o número pode mudar!

In [ ]:
resultados=tabela.sort_values(ascending=False).head(50) #Nomeei a tabela como "resultados"

In [ ]:
resultados #Para vizualizar a tabela de PandaSeries e verificar se está organizada

In [ ]:
s = resultados #Renomeei como "s" é para lembrar que resultados se tratava de uma PandaSeries! Precisei de transformar em Dataframe.

In [ ]:
df = s.to_frame()

In [ ]:
print(df)

In [ ]:
df.columns = ['Vezes que a palavra foi usada']

In [ ]:
from wordcloud import WordCloud #Decidi por uma wordcloud para ilustrar a ferramenta de leitura de discursos.
import matplotlib.pyplot as plt
from IPython.display import Image

In [ ]:
data_str = df.to_string() #Já que a wordcloud só lê strings, transformei a data frame (que antes era uma Panda Series) em uma string.

In [ ]:
data_str

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=50).generate(data_str)

In [ ]:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()
plt.savefig("wordcloud.png")

In [ ]:
mais_usados=resultados.head(5).reset_index()
t1=mais_usados.loc[0, 'index']
t2=mais_usados.loc[1, 'index']
t3=mais_usados.loc[2, 'index']
t4=mais_usados.loc[3, 'index']
t5=mais_usados.loc[4, 'index']

In [ ]:
print(f"Os termos mais utilizados neste discurso na Câmara dos Deputados foram:", t1,',',t2,',',t3,',',t4, 'e',t5 )
print(Image(filename="wordcloud.png"))